In [1]:
from glob import glob as gg
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm as cm
from matplotlib import rc as rc
#
from netCDF4 import Dataset  
#
from datetime import datetime,date
#
import pandas as pd

In [2]:
#
# Parameters, path and filename related to mesh pre-processing
#

# option to create daily mesh (corresponding to daily average observations) or subhourly/hourly mesh
flag_daily=False # set to - True for daily grid output (early preprocessing steps) 
                # or     - False according to the simulation setting and corresponding forcing 
    
# location and name of the time configuration info file for REcoM simulations (basically define the time frame of the simulation)
path_time='../../REcoM1D/config/'
filename_time=path_time + 'time.recom'

# path general to data
path_input='../../data/'

# information related to the type of FESOM mesh we are using for the mesh creation, pre-preprocessing and simulation (farc vs core2)
# and related path and filename of the mesh
mesh_type='farc'
path_input_mesh = path_input + 'MESH/mesh/' + mesh_type + '/'
filename_mesh_aux = path_input_mesh + 'aux3d.out'
filename_mesh_nodes = path_input_mesh + 'nod2d.out'
# information related to the MOSAiC track
path_input_track=path_input + 'TRACK/'
filename_track = path_input_track + 'Polarstern_daily_track.nc'

# define output path
filename_output = path_input + 'MESH/' + 'REcoM1D_daily_mesh.nc' if flag_daily else path_input + 'MESH/' + 'REcoM1D_mesh.nc'

In [3]:
#
# load time information related to the simulation time frame
# infos about start date (spinup excluded), end date of the simulation, time step and spinup time (in days)
#
class REcoM_time:
    def __init__(self, filename, flag_daily):
        # load time properties
        self.estimate_time_axis(filename, flag_daily)
    def estimate_time_axis(self, filename, flag_daily):
        # read data
        with open(filename) as f:
            lines = f.readlines()
        for l in lines:
            tmp = l.replace(' ','').replace('\t','').split('=')[-1].strip()
            # read start date of the simulation
            if 'start_date' in l:
                date_start = datetime.strptime(tmp,'%d-%m-%Y').toordinal()
            # read end date of the simulation
            elif 'end_date' in l:
                date_end = datetime.strptime(tmp,'%d-%m-%Y').toordinal()
            # read time step
            elif 'dt' in l:
                dt = float(tmp)/24.
            # read spinup time
            elif 'spinup' in l:
                spinup = int(tmp)
            if flag_daily:
                dt, spinup = 1., 0
        self.date_start, self.date_end = np.copy(date_start), np.copy(date_end)
        
        # estimate starting date accounting for spinup
        date_start = date_start - spinup
        
        # create time axis
        npt = int(abs(date_end - date_start)/dt)
        self.dates = np.linspace(date_start, date_end, npt+1)        
#
recom_dates = REcoM_time(filename_time, flag_daily)

In [4]:
#
# load vertical mesh information (from aux3d.out from core2 or farc)
#
class vertical_mesh(object):
    def __init__(self, filename):
        self.read_vertical_mesh(filename)
    def read_vertical_mesh(self,filename):
        data = np.asarray(pd.read_csv(filename,header=None))
        # number of depth level
        self.nl = int(data[0])
        # vertical z at the mesh nodes  
        self.zbar = np.reshape(data[1:self.nl+1], len(data[1:self.nl+1]))
        # vertical z at the center of cells
        self.Z = 0.5*(self.zbar[1:] + self.zbar[:-1])
 #       
vmesh = vertical_mesh(filename_mesh_aux)

In [5]:
#
# load MOSAiC track
#
class MOSAiC_track:
    def __init__(self,filename, data):
        self.read_MOSAiC_track(filename, data)
    def read_MOSAiC_track(self,filename, data):
        # open nc file
        ncid = Dataset(filename, "r", format="NETCDF4")        

        # read dates and geo coordinates
        dates = ncid.variables['dates'][:]
        longitude, latitude = ncid.variables['longitude'][:], ncid.variables['latitude'][:]
        # close nc file
        ncid.close()
        # crop data according to simulation time frame
        ind1, ind2 = np.where(dates>=data.date_start)[0], np.where(dates<=data.date_end)[0]
        indices = list(set(ind1) & set(ind2))
        
        # store data
        self.dates = dates[indices]
        self.longitude, self.latitude = longitude[indices], latitude[indices]
#
#filename = path_input_track + 'Polarstern_trajectory.nc'
track=MOSAiC_track(filename_track, recom_dates)

In [6]:
#
# load mesh + depth (truncated in the high latitude)
#
class Mesh:
    def __init__(self, filename_aux, filename_nodes, latmin):
        self.read_mesh(filename_nodes)
        self.read_depth(filename_aux)
        self.truncate_mesh(latmin)
    # read horizontal mesh node coordinates
    def read_mesh(self,filename):
        data = pd.read_csv(filename, sep='\s+', skiprows=1 , header=None)
        self.longitude, self.latitude = data.iloc[:,1], data.iloc[:,2]
    # read mesh vertical level information (depth at nodes)
    def read_depth(self,filename):
        data = np.asarray(pd.read_csv(filename,header=None))
        nb = int(data[0])
        self.depth = data[nb+1:]
    # truncate domain according to minimum latitude (to avoid extensive computation cost)
    def truncate_mesh(self,latmin):
        indices=np.where(self.latitude>=latmin-0.5)[0]
        self.longitude, self.latitude, self.depth = self.longitude[indices], self.latitude[indices], self.depth[indices]
                 
mesh=Mesh(filename_mesh_aux, filename_mesh_nodes, np.min(track.latitude))

In [ ]:
#
# estimate depth along the MOSAiC trajectory
#
def tunnel_distance_nearest_neighbor_index(lon, lat, longitude_ref, latitude_ref):
    #
    # estimate nearest neighbor of (lon,lat) point and return point index
    #
    # convert lon, lat to radian
    deg2rad = np.pi / 180.0
    lon, lat, longitude_ref, latitude_ref =  deg2rad*lon, deg2rad*lat , deg2rad*longitude_ref, deg2rad*latitude_ref
    
    #compute cos,sin and cross-products for of the different a lon,lat arrays
    # grid
    clat, clon, slat, slon = np.cos(lat), np.cos(lon), np.sin(lat), np.sin(lon)
    clat_clon,  clat_slon = clat * clon, clat * slon
    
    # reference node
    clat_ref, clon_ref, slat_ref, slon_ref = np.cos(latitude_ref), np.cos(longitude_ref), np.sin(latitude_ref), np.sin(longitude_ref)
    clat_clon_ref,  clat_slon_ref = clat_ref * clon_ref, clat_ref * slon_ref
    
    # compute tunnel distance to the different grid nodes
    distance = []
    dX, dY, dZ = clat_clon - clat_clon_ref, clat_slon - clat_slon_ref, slat - slat_ref
    distance = dX**2 + dY**2 + dZ**2

    return np.argmin(distance)
# 
class Complete_track_data(object):
    def __init__(self, track, mesh):
        #data
        self.dates, self.longitude, self.latitude = track.dates, track.longitude, track.latitude
        # estimate depth
        self.estimate_depth(mesh)
        
    def estimate_depth(self,mesh):
        depth=[]
        for i in range(len(self.dates)):
            lon, lat = self.longitude[i],self.latitude[i]
            index=tunnel_distance_nearest_neighbor_index(mesh.longitude, mesh.latitude, lon, lat)
            depth.append(mesh.depth[index])
        depth=np.asarray(depth)
        self.depth = np.reshape(depth, depth.size)
        
new_track=Complete_track_data(track,mesh)

In [8]:
#
# include nb of levels in use at each track point 
#
class vertical_track(object):
    def __init__(self,track, vmesh):
        self.dates, self.longitude, self.latitude, self.depth = track.dates, track.longitude, track.latitude, track.depth
        self.estimate_nb_of_level(vmesh)
    def estimate_nb_of_level(self,vmesh):
        # compare depth
        nlevels=[]
        for dpth in self.depth:
            nlevels.append(np.where(vmesh.zbar - dpth <0)[0][0])
        self.nlevels = np.asarray(nlevels)
#
track_data = vertical_track(new_track,vmesh)

In [11]:
#
# estimate coordinate, depth and level at each simulation time using MOSAiC track observations and manage spinup
#
class simulation_data:
    def __init__(self, recom_dates, vmesh, track):
        self.dates = recom_dates.dates
        self.nl, self.zbar, self.Z = vmesh.nl, vmesh.zbar, vmesh.Z
        # estimate coordinates, depth and 
        self.estimate_simulation_mesh_characteristics(recom_dates, track)
        
    def estimate_simulation_mesh_characteristics(self, recom_dates, track):
        #initialization
        lon, lat, dpth, nlvl = [], [], [], []
        # simulation part without spin up
        var = recom_dates
        ind1, ind2 = np.where(var.dates>=var.date_start)[0], np.where(var.dates<=var.date_end)[0]
        indices = list(set(ind1) & set(ind2))
        
        date = var.dates[indices]
        
        for dt in date:    
            # look for corresponding dates
            index = np.argmin(abs(np.floor(track.dates)-np.floor(dt)))
            lon.append(track.longitude[index]), lat.append(track.latitude[index])
            dpth.append(track.depth[index]), nlvl.append(track.nlevels[index])
            # store data
        long, lati, dept, nlev = np.asarray(lon), np.asarray(lat), np.asarray(dpth), np.asarray(nlvl)
        
        # manage spin up and store data
        npt = len(var.dates)
        # spinup (fill with initial conditions)
        longitude, latitude = np.zeros(npt) + long[0], np.zeros(npt) + lati[0]
        depth, nlevels = np.zeros(npt) + dept[0], np.zeros(npt) + nlev[0]
        # actual simulation
        longitude[indices], latitude[indices] = long, lati
        depth[indices], nlevels[indices] = dept, nlev
        
        # store data
        self.longitude, self.latitude, self.depth, self.nlevels = longitude, latitude, depth, nlevels
#
simu_data = simulation_data(recom_dates,vmesh, track_data)

In [12]:
#
# save data to netcdf file
#
class Output(object):
    def __init__(self, filename, data):
        self.write_grid_information(filename,data)
        
    def write_grid_information(self,filename,data):
        # store grid (Lagrangian and vertical) information
        ncid = Dataset(filename, "w", format="NETCDF4") 

        ## define dimensionss
        ncid.createDimension('time', len(data.dates))
        ncid.createDimension('nl', data.nl)
        ncid.createDimension('nl1', data.nl-1)
        dimnl, dimnl1, dimt = ('nl'), ('nl1'), ('time')

        ## create variables
        # dates
        dt = ncid.createVariable('dates', "f8",'time')
        
        # geo coordinates
        lon, lat = ncid.createVariable('longitude', "f8",'time'), ncid.createVariable('latitude', "f8",'time')
        
        # vertical mesh
        zb, z = ncid.createVariable('zbar', "f8",'nl'), ncid.createVariable('Z', "f8",'nl1')
        
        # related depth and number of vertical levels along the track
        dpth, nlvl = ncid.createVariable('depth', "f8",'time'), ncid.createVariable('nlevels', "f8",'time')
        
        
        ## variables attributes
        # description
        dt.description='dates related to python ordinal dates (reference date 01/01/0001)'
        lon.description, lat.description = 'longitude coordinates', 'latitude coordinates'
        zb.description, z.description = 'vertical discretization of depth axis (from ' + mesh_type + ' mesh)', 'vertical discretization of depth axis (middle of cells)'
        dpth.description, nlvl.description = 'depth (from topography)', 'number of level considered in simulation (according to depth)'
        # units
        dt.units='days'
        lon.units, lat.units = 'degree (east)', 'degree (north)'
        zb.units, z.units = 'm', 'm'
        dpth.units, nlvl.units = 'm', ''        
        
        ## fill variables
        # dates
        ncid['dates'][:] = data.dates
        
        # geo coordinates
        ncid['longitude'][:] , ncid['latitude'][:] =  data.longitude, data.latitude
        
        # vertical mesh
        ncid['zbar'][:] , ncid['Z'][:] = data.zbar, data.Z
        
        # depth and vertical levels along the track
        ncid['depth'][:] , ncid['nlevels'][:] = data.depth, data.nlevels
        
        #
        ncid.close()

#
Output(filename_output, simu_data)